In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from aac_crud_2 import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "abc123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))

fig = px.bar(df, x='breed', y='age_upon_outcome_in_weeks')
fig.update_traces(marker_color='green')
fig.update_layout(width=700, height=500, bargap=0.05)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'grazioso_salvare_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#     html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div([
        html.A([
          html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                    'height': '100px',
                    'width': '100px'
                })
        ], href='https://www.snhu.edu')

    ], style={'textAlign': 'center'}),

   # unique identifier
    html.Div([
        html.Div(children='Kevin Schroeder',
                style = {'textAlign' : 'center'}
        )],
        className='col-8',
        style = {'padding-top' : '1%'}
    ),
    html.Hr(),
    # Interactive dropdown filter
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label':'Water Rescue','value':'water'},
            {'label':'Mountain or Wilderness Rescue','value':'mtn'},
            {'label':'Disaster or Individual Tracking','value':'disaster'},
        ],
        placeholder="Filter by Animal Type",
        searchable=True,
        multi=False,
        style={'width':'90%'}
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        # sorting
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        # pagination
        page_current=0,
        page_size=10,
        # horizontal scroll
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '100px', 'width': '180px', 'maxWidth': '300px',
            'whiteSpace': 'normal'
        },
        # tooltip if data is too large for cell
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        tooltip_duration=None
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(
                id='graph-id',
                figure=fig
                ),
            className='col s12 m6',            
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            # apply styling to map
            style={'marginBottom': 50, 'marginTop': 50, 'minHeight': '400px', 'textAlign': 'center'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'), 
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(value):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    if value == 'water':
        df = pd.DataFrame.from_records(shelter.readAll( 
              {
                'breeds': [ 'Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland' ], 
                'sex': 'Intact Female', 
                'young_week': 26, 
                'old_week': 156 
              }))
        
    elif value == 'mtn':
        df = pd.DataFrame.from_records(shelter.readAll( 
            {
                'breeds':  [ 'German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler' ], 
                'sex': 'Intact Male', 
                'young_week': 26, 
                'old_week': 156 
            }))
    elif value == 'disaster':
        df = pd.DataFrame.from_records(shelter.readAll( 
            {
                'breeds':  [ 'Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler' ],
                'sex': 'Intact Male', 
                'young_week': 20, 
                'old_week': 300 
            }))
    else:
        df = pd.DataFrame.from_records(shelter.readAll({}))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')        
    return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "figure"),
    [Input('filter-type', 'value')])
def update_graphs(value):
    if value == 'water':
        df = pd.DataFrame.from_records(shelter.readAll( 
              {
                'breeds': [ 'Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland' ], 
                'sex': 'Intact Female', 
                'young_week': 26, 
                'old_week': 156 
              }))
        
    elif value == 'mtn':
        df = pd.DataFrame.from_records(shelter.readAll( 
            {
                'breeds':  [ 'German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler' ], 
                'sex': 'Intact Male', 
                'young_week': 26, 
                'old_week': 156 
            }))
    elif value == 'disaster':
        df = pd.DataFrame.from_records(shelter.readAll( 
            {
                'breeds':  [ 'Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler' ],
                'sex': 'Intact Male', 
                'young_week': 20, 
                'old_week': 300 
            }))
    else:
        df = pd.DataFrame.from_records(shelter.readAll({}))

    fig = px.bar(df, x='breed', y='age_upon_outcome_in_weeks')
    fig.update_traces(marker_color='green')
    fig.update_layout(width=700, height=500, bargap=0.05)
    return fig

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app